In [1]:
# --- Podstawowe biblioteki ---
import pandas as pd
import numpy as np
from datetime import datetime
import os

# --- Biblioteki do Machine Learningu i śledzenia eksperymentów ---
from pycaret.regression import setup, compare_models, tune_model, finalize_model, save_model, predict_model, pull

In [2]:
# --- Wczytywanie Danych ---

# 1. Wczytanie głównego zbioru danych
try:
    print("Wczytuję kompletny plik danych: Data_state_LSTM_predicted_full.csv...")
    df_original = pd.read_csv('Data_state_LSTM_predicted_full.csv', sep=';', low_memory=False)
    print(f"Wczytano pomyślnie. Kształt danych: {df_original.shape}")
except Exception as e:
    print(f"BŁĄD: Nie można wczytać głównego pliku danych. {e}")
    df_original = pd.DataFrame()

# 2. Wczytanie słownika hierarchicznego
try:
    print("\nWczytuję słownik hierarchiczny: slownik_finalny_z_hierarchia.csv...")
    df_slownik = pd.read_csv('slownik_finalny_z_hierarchia.csv', sep=';')
    print(f"Słownik wczytany pomyślnie. Kształt danych: {df_slownik.shape}")
except Exception as e:
    print(f"BŁĄD: Nie można wczytać pliku słownika. {e}")
    df_slownik = pd.DataFrame()

# 3. Wczytanie pliku z predykcjami lokalizacji z modelu LSTM
try:
    print("\nWczytuję predykcje lokalizacji: Saleflats_Loc.csv...")
    df_loc_predictions = pd.read_csv('Saleflats_Loc.csv', sep=';')
    # Wybieramy tylko kluczowe kolumny, aby uniknąć duplikatów
    df_loc_predictions = df_loc_predictions[['SaleId', 'Predict_Loc']]
    print(f"Predykcje lokalizacji wczytane pomyślnie. Kształt danych: {df_loc_predictions.shape}")
except Exception as e:
    print(f"BŁĄD: Nie można wczytać pliku z predykcjami lokalizacji. {e}")
    df_loc_predictions = pd.DataFrame()

# --- Łączenie predykcji z głównym zbiorem danych ---
if not df_original.empty and not df_loc_predictions.empty:
    print("\nŁączę predykcje lokalizacji z głównym zbiorem danych...")
    # Używamy left merge, aby zachować wszystkie wiersze z df_original.
    # Wiersze spoza Mazowieckiego (nieobecne w df_loc_predictions) otrzymają NaN w kolumnie Predict_Loc.
    df_original = pd.merge(df_original, df_loc_predictions, on='SaleId', how='left')
    print(f"Połączenie zakończone. Nowy kształt danych: {df_original.shape}")
    print(f"Liczba wierszy z przypisaną predykcją lokalizacji: {df_original['Predict_Loc'].notna().sum()}")
else:
    # Jeśli plik predykcji nie został wczytany, tworzymy pustą kolumnę, aby reszta kodu działała
    if 'Predict_Loc' not in df_original.columns:
        df_original['Predict_Loc'] = np.nan

Wczytuję kompletny plik danych: Data_state_LSTM_predicted_full.csv...
Wczytano pomyślnie. Kształt danych: (760765, 55)

Wczytuję słownik hierarchiczny: slownik_finalny_z_hierarchia.csv...
Słownik wczytany pomyślnie. Kształt danych: (747, 4)

Wczytuję predykcje lokalizacji: Saleflats_Loc.csv...
Predykcje lokalizacji wczytane pomyślnie. Kształt danych: (235700, 2)

Łączę predykcje lokalizacji z głównym zbiorem danych...
Połączenie zakończone. Nowy kształt danych: (760765, 56)
Liczba wierszy z przypisaną predykcją lokalizacji: 92076


In [3]:
# Funkcja do przygotowania danych, która będzie używana do treningu i finalnej predykcji
def prepare_data_for_modeling(input_df, slownik_df):
    """
    Kompleksowo przygotowuje dane: inflacja, typy, wiek budynku, łączenie ze słownikiem
    i tworzenie zunifikowanej kolumny lokalizacyjnej.
    """
    df = input_df.copy()
    
    # 1. Inżynieria Cech (Inflacja)
    if 'AdjustedPrice' not in df.columns:
        print("Tworzenie kolumny 'AdjustedPrice'...")
        def adjust_price(row):
            price = pd.to_numeric(row['Price'], errors='coerce')
            if pd.isna(price): return np.nan
            date_str = row['NewestDate'] if 'NewestDate' in row and pd.notna(row['NewestDate']) else (row['DateAdded'] if 'DateAdded' in row and pd.notna(row['DateAdded']) else None)
            if date_str is None or date_str == 'NULL': return price
            try:
                offer_date = pd.to_datetime(date_str, errors='coerce')
                if pd.isna(offer_date): return price
                years_diff = (datetime.now() - offer_date).days / 365.25
                return round(price * (1.05**years_diff), 0) if years_diff > 0 else price
            except: return price
        df['AdjustedPrice'] = df.apply(adjust_price, axis=1)

    # 2. Konwersja typów i tworzenie 'BuildingAge'
    for col in ['Area', 'NumberOfRooms', 'Floor', 'Floors', 'StreetNumber']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    if 'BuiltYear' in df.columns:
        print("Konwertowanie 'BuiltYear' i tworzenie 'BuildingAge'...")
        median_year = pd.to_numeric(df['BuiltYear'], errors='coerce').median()
        df['BuiltYear_numeric'] = pd.to_numeric(df['BuiltYear'], errors='coerce').fillna(median_year)
        df['BuildingAge'] = datetime.now().year - df['BuiltYear_numeric']
        df['BuiltYear'] = df['BuiltYear_numeric'].astype(int).astype(str)
        df['BuiltYear'] = pd.to_datetime(df['BuiltYear'], format='%Y', errors='coerce')
        df.drop(columns=['BuiltYear_numeric'], inplace=True)

    # 3. Integracja danych ze słownika i czyszczenie
    print("Integrowanie danych lokalizacyjnych ze słownika...")
    if 'StreetNumber' in df.columns and 'UlicaID' in slownik_df.columns:
        df.rename(columns={'StreetNumber': 'UlicaID'}, inplace=True, errors='ignore')
        df = pd.merge(df, slownik_df, on='UlicaID', how='left')
        for col in ['Dzielnica_Name', 'Ulica_Name']:
             if col in df.columns:
                df[col].fillna('Brak Danych', inplace=True)

    # 4. Czyszczenie kluczowych cech kategorycznych
    if 'Predict_State' in df.columns:
        df['Predict_State'].fillna('Brak Danych', inplace=True)
    if 'Predict_Loc' not in df.columns:
        df['Predict_Loc'] = np.nan

    # 5. Tworzenie zunifikowanej kolumny lokalizacyjnej
    print("Tworzenie zunifikowanej kolumny 'Unified_Location'...")
    location_str_fallback = df['Dzielnica_Name'] + ' > ' + df['Ulica_Name']
    df['Unified_Location'] = np.where(df['Predict_Loc'].notna(), df['Predict_Loc'], location_str_fallback)
    
    print("Przetwarzanie danych zakończone.")
    return df

# --- Główny przepływ ---
if not df_original.empty and not df_slownik.empty:
    df_processed = prepare_data_for_modeling(df_original, df_slownik)
    
    data_for_training = df_processed.dropna(subset=['AdjustedPrice', 'Area']).copy()
    print(f"\nLiczba wierszy (przed czyszczeniem outlierów): {len(data_for_training)}")

    for col in ['AdjustedPrice', 'Area']:
        Q1 = data_for_training[col].quantile(0.25)
        Q3 = data_for_training[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound, upper_bound = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
        data_for_training = data_for_training[(data_for_training[col] >= lower_bound) & (data_for_training[col] <= upper_bound)]
    
    print(f"Liczba wierszy do treningu (po czyszczeniu): {len(data_for_training)}")
else:
    print("BŁĄD: Zbiór 'df_original' lub 'df_slownik' jest pusty. Przerwanie przetwarzania.")
    data_for_training = pd.DataFrame()

Tworzenie kolumny 'AdjustedPrice'...
Konwertowanie 'BuiltYear' i tworzenie 'BuildingAge'...
Integrowanie danych lokalizacyjnych ze słownika...
Tworzenie zunifikowanej kolumny 'Unified_Location'...
Przetwarzanie danych zakończone.

Liczba wierszy (przed czyszczeniem outlierów): 760765
Liczba wierszy do treningu (po czyszczeniu): 692616


In [4]:
# NOWA KOMÓRKA - NLP Feature Engineering
from sklearn.feature_extraction.text import TfidfVectorizer

if not data_for_training.empty:
    print("Rozpoczynam tworzenie cech z opisów (TF-IDF)...")
    
    # 1. Wypełnij puste opisy i oczyść tekst
    data_for_training['Description_clean'] = data_for_training['Description'].fillna('').str.lower()
    
    # 2. Skonfiguruj wektoryzator TF-IDF
    # Wybieramy 100 najważniejszych słów/fraz, ignorując popularne polskie spójniki
    vectorizer = TfidfVectorizer(max_features=100, stop_words=['i', 'w', 'na', 'z', 'do', 'jest', 'oraz', 'mieszkanie', 'mieszkania', 'zl'])
    
    # 3. Dopasuj i przekształć opisy na cechy numeryczne
    tfidf_features = vectorizer.fit_transform(data_for_training['Description_clean'])
    
    # 4. Konwertuj na DataFrame i dodaj do głównego zbioru
    df_tfidf = pd.DataFrame(tfidf_features.toarray(), columns=[f"tfidf_{name}" for name in vectorizer.get_feature_names_out()])
    df_tfidf.index = data_for_training.index # Dopasuj indeksy, aby uniknąć błędów
    
    data_for_training = pd.concat([data_for_training, df_tfidf], axis=1)
    
    print(f"Dodano {df_tfidf.shape[1]} nowych cech z opisów.")
    print(f"Nowy kształt danych do treningu: {data_for_training.shape}")
else:
    print("Zbiór 'data_for_training' jest pusty, pomijam NLP.")

Rozpoczynam tworzenie cech z opisów (TF-IDF)...
Dodano 100 nowych cech z opisów.
Nowy kształt danych do treningu: (692616, 163)


In [5]:
# NOWA KOMÓRKA - ZAPIS WEKTORYZATORA
import joblib

# Zapisujemy obiekt wektoryzatora do pliku, aby móc go użyć w skrypcie predykcyjnym
vectorizer_filename = 'tfidf_vectorizer.pkl'
joblib.dump(vectorizer, vectorizer_filename)

print(f"Wektoryzator TF-IDF został pomyślnie zapisany do pliku: {vectorizer_filename}")

Wektoryzator TF-IDF został pomyślnie zapisany do pliku: tfidf_vectorizer.pkl


In [6]:
if not data_for_training.empty:
    print("--- Konfiguracja eksperymentu PyCaret ---\n")
    
    # Cechy numeryczne teraz zawierają wiek budynku. TF-IDF zostanie wykryte automatycznie.
    numeric_features = ['Area', 'NumberOfRooms', 'Floor', 'Floors', 'BuildingAge']
    date_features = ['BuiltYear']
    
    # Cechy kategoryczne zawierają teraz Predict_State oraz Unified_Location
    categorical_features = ['BuildingType', 'TypeOfMarket', 'Type', 'OfferFrom', 'OwnerType', 'Predict_State', 'Unified_Location']

    print(f"Finalna lista cech kategorycznych do użycia w modelu: {categorical_features}\n")
    
    ignore_features = [
        'SaleId', 'OriginalId', 'PortalId', 'Title', 'Description', 'Description_clean', # Ignorujemy surowy tekst
        'BuildingCondition', 'Price',
        'DateAddedToDatabase', 'DateAdded', 'NewestDate', 'DateLastModification', 'DateLastRaises',
        'OfferPrice', 'RealPriceAfterRenovation', 'OriginalPrice', 'PricePerSquareMeter',
        'AvailableFrom', 'Link', 'Phone', 'MainImage', 'OtherImages', 'NumberOfDuplicates',
        'NumberOfRaises', 'NumberOfModifications', 'IsDuplicatePriceLower', 'IsDuplicatePrivateOwner',
        'Score', 'ScorePrecision', 'CommunityScore', 'NumberOfCommunityComments', 'NumberOfCommunityOpinions',
        'Archive', 'Location', 'VoivodeshipNumber', 'CountyNumber', 'CommunityNumber', 'KindNumber',
        'RegionNumber', 'SubRegionNumber', 'StreetNumber', 'EncryptedId', 'PredictedRenovation',
        'LocationPath', 'UlicaID', 'Predict_Loc', 
        'MiastoID', 'DzielnicaID', 'Wojewodztwo_Name', 'Miasto_Name', 'Dzielnica_Name', 'Ulica_Name'
    ]

    numeric_features_to_use = [c for c in numeric_features if c in data_for_training.columns]
    categorical_features_to_use = [c for c in categorical_features if c in data_for_training.columns]
    date_features_to_use = [c for c in date_features if c in data_for_training.columns]
    ignore_features_to_use = [c for c in ignore_features if c in data_for_training.columns]

    print(f"Cechy numeryczne: {numeric_features_to_use}")
    print(f"Cechy kategoryczne: {categorical_features_to_use}")
    print(f"Cechy daty: {date_features_to_use}")
    print(f"Liczba ignorowanych cech: {len(ignore_features_to_use)}")

    # Inicjalizacja środowiska PyCaret
    reg_exp = setup(
        data=data_for_training,
        target='AdjustedPrice',
        session_id=123,
        log_experiment=False, 
        numeric_features=numeric_features_to_use,
        categorical_features=categorical_features_to_use,
        date_features=date_features_to_use,
        ignore_features=ignore_features_to_use,
        normalize=True,
        normalize_method='zscore'
    )
else:
    print("BŁĄD: Zbiór 'data_for_training' jest pusty. Nie można uruchomić setup().")

--- Konfiguracja eksperymentu PyCaret ---

Finalna lista cech kategorycznych do użycia w modelu: ['BuildingType', 'TypeOfMarket', 'Type', 'OfferFrom', 'OwnerType', 'Predict_State', 'Unified_Location']

Cechy numeryczne: ['Area', 'NumberOfRooms', 'Floor', 'Floors', 'BuildingAge']
Cechy kategoryczne: ['BuildingType', 'TypeOfMarket', 'Type', 'OfferFrom', 'OwnerType', 'Predict_State', 'Unified_Location']
Cechy daty: ['BuiltYear']
Liczba ignorowanych cech: 48


,Description,Value
0,Session id,123
1,Target,AdjustedPrice
2,Target type,Regression
3,Original data shape,"(692616, 163)"
4,Transformed data shape,"(692616, 123)"
5,Transformed train set shape,"(484831, 123)"
6,Transformed test set shape,"(207785, 123)"
7,Ignore features,48
8,Numeric features,5
9,Date features,1


In [7]:
# --- Bezpieczne Porównanie Modeli ---

if 'reg_exp' in locals() and reg_exp is not None:
    print("Rozpoczynam porównywanie wszystkich dostępnych modeli. To może potrwać...")
    best_models_list = compare_models(sort='R2', n_select=3)

    if best_models_list:
        best_model = best_models_list[0]
        print("\n--- Najlepsze znalezione modele ---")
        print(best_models_list)
        print("\n--- Wybrany najlepszy model do dalszej pracy ---")
        print(best_model)
    else:
        print("\nBŁĄD: compare_models nie zwróciło żadnych modeli.")
        best_model = None
else:
    print("BŁĄD KRYTYCZNY: Eksperyment PyCaret (setup) nie został uruchomiony.")
    best_model = None

Rozpoczynam porównywanie wszystkich dostępnych modeli. To może potrwać...


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,87399.3818,16705860627.2814,129248.9099,0.7211,0.4380,44.9359,123.6420
catboost,CatBoost Regressor,104085.2953,20085913865.6316,141722.7209,0.6647,0.4519,43.8223,14.2070
xgboost,Extreme Gradient Boosting,107231.2609,21126875545.6000,145348.6812,0.6473,0.4567,43.2812,6.2900
lightgbm,Light Gradient Boosting Machine,113170.9944,22994327250.1354,151636.9562,0.6161,0.4663,43.5963,3.9470
dt,Decision Tree Regressor,115870.6967,33717258541.3927,183620.5696,0.4370,0.6027,42.6736,7.2860
dummy,Dummy Regressor,193472.0469,59896643584.0000,244737.0812,-0.0000,0.6167,57.5122,1.2700



--- Najlepsze znalezione modele ---
[RandomForestRegressor(n_jobs=-1, random_state=123), <catboost.core.CatBoostRegressor object at 0x0000023D268A6AD0>, XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cpu', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=-1, num_parallel_tree=None, ...)]

--- Wybrany najlepszy model do dalszej pracy ---
RandomForestRegressor(n_jobs=-1, random_st

In [8]:
# --- KROK: TUNE_MODEL ---

if 'best_model' in locals() and best_model is not None:
    print(f"Rozpoczynam tuning najlepszego modelu: {type(best_model).__name__}")
    tuned_best_model = tune_model(best_model, n_iter=20, optimize = 'R2')
    
    print("\n--- Wyniki po tuningu ---")
    tuned_results = pull()
    display(tuned_results)
    
    best_model = tuned_best_model 
else:
    print("BŁĄD: Zmienna 'best_model' nie została znaleziona lub jest pusta.")

Rozpoczynam tuning najlepszego modelu: RandomForestRegressor


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,126722.1215,28423860864.7886,168593.7747,0.5279,0.5043,67.2943
1,125557.3681,27987359793.5811,167294.2312,0.5340,0.5018,41.4533
2,125350.1916,27732469387.0083,166530.6860,0.5352,0.4836,59.0807
3,125696.4065,27764087234.4200,166625.5900,0.5349,0.4811,19.1129
4,125467.4590,27678914494.6803,166369.8125,0.5414,0.4737,15.3731
5,125927.7279,27887310292.6660,166994.9409,0.5362,0.4943,44.3988
6,125236.9565,27668613062.0033,166338.8501,0.5335,0.5165,60.0515
7,126420.1114,28258498187.6877,168102.6418,0.5281,0.5032,33.8937
8,125494.9260,27891896008.5952,167008.6705,0.5358,0.5060,78.0054


Fitting 10 folds for each of 20 candidates, totalling 200 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).

--- Wyniki po tuningu ---


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,126722.1215,2.842386e+10,168593.7747,0.5279,0.5043,67.2943
1,125557.3681,2.798736e+10,167294.2312,0.5340,0.5018,41.4533
2,125350.1916,2.773247e+10,166530.6860,0.5352,0.4836,59.0807
3,125696.4065,2.776409e+10,166625.5900,0.5349,0.4811,19.1129
4,125467.4590,2.767891e+10,166369.8125,0.5414,0.4737,15.3731
5,125927.7279,2.788731e+10,166994.9409,0.5362,0.4943,44.3988
6,125236.9565,2.766861e+10,166338.8501,0.5335,0.5165,60.0515
7,126420.1114,2.825850e+10,168102.6418,0.5281,0.5032,33.8937
8,125494.9260,2.789190e+10,167008.6705,0.5358,0.5060,78.0054


In [9]:
# --- Finalizacja, Zapis i Predykcja ---

if 'best_model' in locals() and best_model is not None:
    print("Finalizowanie najlepszego modelu...")
    final_model = finalize_model(best_model)
    
    # WAŻNE: Musimy też przetworzyć dane do predykcji przez TF-IDF
    # Kopiujemy całą logikę z nowej komórki NLP
    print("\nPrzygotowuję oryginalny zbiór danych do predykcji...")
    data_for_prediction = prepare_data_for_modeling(df_original, df_slownik)
    
    print("Stosowanie transformacji TF-IDF na danych do predykcji...")
    data_for_prediction['Description_clean'] = data_for_prediction['Description'].fillna('').str.lower()
    # UWAGA: Używamy .transform() a nie .fit_transform()!
    tfidf_features_pred = vectorizer.transform(data_for_prediction['Description_clean'])
    df_tfidf_pred = pd.DataFrame(tfidf_features_pred.toarray(), columns=[f"tfidf_{name}" for name in vectorizer.get_feature_names_out()])
    df_tfidf_pred.index = data_for_prediction.index
    data_for_prediction = pd.concat([data_for_prediction, df_tfidf_pred], axis=1)

    if 'AdjustedPrice' in data_for_prediction.columns:
        data_for_prediction_clean = data_for_prediction.drop(columns=['AdjustedPrice'])
    else:
        data_for_prediction_clean = data_for_prediction
    
    all_data_predictions = predict_model(final_model, data=data_for_prediction_clean)
    print("\nPredykcja zakończona.")

    all_data_predictions.rename(columns={'prediction_label': 'PredictedPrice'}, inplace=True)
    
    df_final_output = df_original.copy()
    df_final_output['AdjustedPrice'] = data_for_prediction['AdjustedPrice']
    df_final_output['PredictedPrice'] = all_data_predictions['PredictedPrice']
    
    output_filename = 'sale_2024_0_predict.csv'
    df_final_output.to_csv(output_filename, index=False, sep=';', decimal='.')

    print(f"\nWyniki zostały zapisane do pliku: {output_filename}")
    print("\nPrzykładowe dane z finalnego pliku:")
    display(df_final_output[['SaleId', 'Price', 'AdjustedPrice', 'PredictedPrice']].head(10))
else:
    print("BŁĄD: Zmienna 'best_model' nie została znaleziona lub jest pusta.")

Finalizowanie najlepszego modelu...

Przygotowuję oryginalny zbiór danych do predykcji...
Tworzenie kolumny 'AdjustedPrice'...
Konwertowanie 'BuiltYear' i tworzenie 'BuildingAge'...
Integrowanie danych lokalizacyjnych ze słownika...
Tworzenie zunifikowanej kolumny 'Unified_Location'...
Przetwarzanie danych zakończone.
Stosowanie transformacji TF-IDF na danych do predykcji...



Predykcja zakończona.

Wyniki zostały zapisane do pliku: sale_2024_0_predict.csv

Przykładowe dane z finalnego pliku:


,SaleId,Price,AdjustedPrice,PredictedPrice
0,88,766500.0,826920.0,811744.144667
1,99,540000.0,580236.0,575440.555000
2,115,540000.0,579771.0,583329.776667
3,140,544000.0,567076.0,651574.860000
4,145,459000.0,491294.0,485109.630000
5,159,779000.0,835146.0,786704.810000
6,165,359000.0,386110.0,384263.540000
7,173,380000.0,406844.0,396061.150000
8,189,354000.0,379312.0,377007.870000
9,208,820000.0,880746.0,572209.458000
